In [1]:
import ipywidgets as widgets
from IPython.display import display
import asyncio
import wave
import struct
from enum import Enum


log = widgets.Output(layout={"border": "1px solid black"})


def info(str):
    log.append_stdout(str + "\n")


clear_log_button = widgets.Button(description="Clear")
clear_log_button.on_click(lambda b: log.clear_output())
log_container = widgets.VBox([log, clear_log_button])

start_button = widgets.Button(description="Start Server")

stop_button = widgets.Button(description="Stop Server")
stop_button.disabled = True

clients_container = widgets.VBox([])

ui = widgets.VBox(
    [widgets.HBox([start_button, stop_button]), log_container, clients_container]
)

clients_ui = {}


def redraw_clients(new: tuple[str, any] = None, remove=None):
    if new:
        clients_ui[new[0]] = new[1]

    if remove:
        clients_ui.pop(remove, None)

    clients_container.children = tuple(clients_ui.values())

In [2]:
class State(Enum):
    STOPPED = 1
    RECORDING = 2
    DISCONNECTED = 3
    PAUSED = 4


class ButtonType(Enum):
    RECORD = 1
    SAVE = 2
    DISCONNECT = 3
    PAUSE = 4


state = {}
clients = {}
buttons = {}


def save(address):
    channels, sample_width, frame_rate, sound = clients[address]
    file_name = f"output_{address[0]}_{address[1]}.wav"
    save_wav(file_name, channels, sample_width, frame_rate, sound)
    info(f"Audio saved to {file_name}")


def set_state(address, new_state):
    if new_state == State.STOPPED:
        buttons[address][ButtonType.RECORD].disabled = False
        buttons[address][ButtonType.SAVE].disabled = True
        buttons[address][ButtonType.DISCONNECT].disabled = True
        buttons[address][ButtonType.PAUSE].disabled = True
    elif new_state == State.RECORDING:
        buttons[address][ButtonType.RECORD].disabled = True
        buttons[address][ButtonType.SAVE].disabled = False
        buttons[address][ButtonType.DISCONNECT].disabled = False
        buttons[address][ButtonType.PAUSE].disabled = False
    elif new_state == State.DISCONNECTED:
        buttons[address][ButtonType.RECORD].disabled = True
        buttons[address][ButtonType.SAVE].disabled = True
        buttons[address][ButtonType.DISCONNECT].disabled = True
        buttons[address][ButtonType.PAUSE].disabled = True
    elif new_state == State.PAUSED:
        buttons[address][ButtonType.RECORD].disabled = False
        buttons[address][ButtonType.SAVE].disabled = False
        buttons[address][ButtonType.DISCONNECT].disabled = False
        buttons[address][ButtonType.PAUSE].disabled = True

    state[address] = new_state


async def handle_new_client(reader, writer):
    address = writer.get_extra_info("peername")

    record_button = widgets.Button(description="Record")
    record_button.on_click(lambda b: set_state(address, State.RECORDING))

    save_button = widgets.Button(description="Save")
    save_button.on_click(lambda b: save(address))

    disconnect_button = widgets.Button(description="Disconnect")
    disconnect_button.on_click(lambda b: set_state(address, State.DISCONNECTED))

    pause_button = widgets.Button(description="Pause")
    pause_button.on_click(lambda b: set_state(address, State.PAUSED))

    buttons[address] = {
        ButtonType.RECORD: record_button,
        ButtonType.SAVE: save_button,
        ButtonType.DISCONNECT: disconnect_button,
        ButtonType.PAUSE: pause_button,
    }

    name = widgets.Label(value=str(address))
    client = widgets.HBox(
        [name, record_button, pause_button, save_button, disconnect_button]
    )

    set_state(address, State.STOPPED)

    redraw_clients(new=(address, client))

    await handle_client(reader, writer)


async def handle_client(reader, writer):
    address = writer.get_extra_info("peername")

    info(f"Connected by {address}")

    # Receive audio configuration
    config_data = b""
    while len(config_data) < 12:
        config_data += await reader.read(12 - len(config_data))
    channels, sample_width, frame_rate = struct.unpack("III", config_data)

    clients[address] = (channels, sample_width, frame_rate, b"")
    try:
        while True:
            data_size_bytes = b""
            while len(data_size_bytes) < 2:
                data_size_bytes += await reader.read(2 - len(data_size_bytes))
            data_size = struct.unpack("H", data_size_bytes)[0]

            data_chunk = b""
            while len(data_chunk) < data_size:
                data_chunk += await reader.read(data_size - len(data_chunk))

            # TODO: better backpressure
            if state[address] == State.RECORDING:
                sound = clients[address][3]
                clients[address] = (
                    channels,
                    sample_width,
                    frame_rate,
                    sound + data_chunk,
                )

            if state[address] == State.DISCONNECTED:
                info(f"Disconnected from {address}")
                break
    except asyncio.CancelledError:
        info(f"Connection to {address} was cancelled")
    except ConnectionResetError:
        info(f"Connection to {address} was reset by the peer")
    except Exception as e:
        info(f"An error occurred with {address}: {e}")
    finally:
        writer.close()

        redraw_clients(remove=address)

        await writer.wait_closed()


def save_wav(file_name, channels, sample_width, frame_rate, data):
    with wave.open(file_name, mode="wb") as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(frame_rate)
        wav_file.writeframes(data)


In [5]:
HOST = "127.0.0.1"
PORT = 3000


async def start_server():
    global server
    server = await asyncio.start_server(handle_new_client, HOST, PORT)
    addr = server.sockets[0].getsockname()
    info(f"Serving on {addr}")
    start_button.disabled = True
    stop_button.disabled = False


async def stop_server():
    global server
    if server:
        server.close()
        await server.wait_closed()
        info("Server stopped")
        start_button.disabled = False
        stop_button.disabled = True


start_button.on_click(lambda b: asyncio.create_task(start_server()))
stop_button.on_click(lambda b: asyncio.create_task(stop_server()))

In [4]:
display(ui)